#ML model Hyperparameter Tuning
## Loading the CSV dataset in the DBFS (Databricks File System)

In [0]:
%sh
 rm -r /dbfs/mlflow_lab
 mkdir /dbfs/mlflow_lab
 wget -O /dbfs/mlflow_lab/diabetes.csv https://raw.githubusercontent.com/kuljotSB/DatabricksUdemyCourse/refs/heads/main/MachineLearningModel/diabetes.csv